In [1]:
import textacy
import textacy.representations.vectorizers as vectorizers
import pickle
import numpy as np
import spacy
nlp = spacy.load("en_core_web_trf")


/home/dennislai/anaconda3/envs/P76I900/lib/python3.9/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:115.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
vectorizer = pickle.load(open("../../models/P76I900/HW4/tfidf_vectorizer.pkl", "rb"))
doc_matrix = pickle.load(open("../../models/P76I900/HW4/tfidf_doc_matrix.pkl", "rb"))

In [3]:
corpus_test = textacy.Corpus.load("en_core_web_trf", "../../models/P76I900/HW4/wikinews_4cls_corpus_train.bin.gz")

In [18]:
def get_sents(doc):
    doc = nlp(doc)
    return list(doc.sents)

In [25]:

def simple_rank_sentences(doc, vec, doc_matrix, topn=3):

    sents = get_sents(doc)
    sent_idx = [[vec.vocabulary_terms[w.lemma_] for w in sent if w.lemma_ in vec.vocabulary_terms] for sent in sents]
    tfidf_sents = [[doc_matrix[:, idx].toarray() for idx in sent] for sent in sent_idx]
    doc_val = np.sum(doc_matrix)
    sent_vals = [(float(np.sum(sent)) / doc_val) for sent in tfidf_sents]
    ranked = [p for p in zip(range(len(sent_vals)), sent_vals)]
    ranked = sorted(ranked, key=lambda x: x[1], reverse=True)

    return ranked[:topn]

In [26]:
test_doc = corpus_test[0]
ranked = simple_rank_sentences(test_doc, vectorizer, doc_matrix)

In [27]:
ranked

[(10, 0.01419279933712399),
 (9, 0.010805623696357443),
 (12, 0.010679161843208954)]

In [28]:
sents = get_sents(test_doc)
for idx, v in ranked:
    print(sents[idx], ": ", v, "\n")
print(test_doc)

"I'm grateful this ordeal is over, and I'm so looking forward to getting back to the job I love," Abdul said in an Associated Press interview after the decision was announced to keep her on the show. :  0.01419279933712399 

Two years later, he told reporters that he had been involved in a secret romantic relationship with Abdul, which would have been a violation of the show's rules and Abdul's employment contract. :  0.010805623696357443 

"'Idol' Probe Finds No Proof Of Paula Abdul/ Corey Clark Affair" — MTV News, August 12, 2005 "Paula Abdul to remain on 'American Idol,' Fox says" — Associated Press, August 13, 2005 :  0.010679161843208954 

Sunday, August 14, 2005  Executives of the U.S. Fox network said Friday that singer Paula Abdul was cleared of any wrongdoing in regard to charges that she had an improper relationship with former American Idol contestant Corey Clark. The network cleared her to return to the show for its fifth season. American Idol has topped network television 